In [26]:
from segmentCopy import multiprocess_segmentation, track_cyto_intensities
from metadataCopy import Metadata
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import os
from skimage.exposure import equalize_adapthist
import napari
from segmentCopy import create_nuc_cyto_visualization
base = '/bigstore/GeneralStorage/Evan/MAPK_FISH/Experiment for determining optimal concentrations of ligands/96_well_EGF_mIL3_2020Sep25_V2/'#Extra_Conservative_Rings/'
conservative_base = '/bigstore/GeneralStorage/Evan/MAPK_FISH/Experiment for determining optimal concentrations of ligands/96_well_EGF_mIL3_2020Sep25_V2/Extra_Conservative_Rings/'

ModuleNotFoundError: No module named 'skimage'

In [22]:
import skimage

ModuleNotFoundError: No module named 'skimage'

In [23]:
md = Metadata('/bigstore/Images2020/Maeve/MAPK_FISH/96_well_EGF_mIL3_2020Sep25')
positions  = md.posnames

NameError: name 'Metadata' is not defined

In [24]:
positions

NameError: name 'positions' is not defined

In [25]:
track_cyto_intensities('C02', conservative_base+'Long_Term_Test/', 
                       cytoplasmic_channels={'ERK KTR': 'Yellow', 'JNK KTR': 'Orange'}, 
                       metadata=md, nuclear_channel='FarRed', memory=2, stub_length=2, 
                       min_track_length=4, keep_frames=[i for i in range(4,56)], nuc_cyto_ring=True, report=True)

NameError: name 'track_cyto_intensities' is not defined

In [ ]:
multiprocess_segmentation(positions=positions, save_path=base, nuclear_channel='FarRed',
                          cytoplasmic_channels={'ERK KTR': 'Yellow', 'JNK KTR': 'Orange'}, 
                          metadata=md, keep_frames=[0,1,2,3], memory=2, stub_length=2, min_track_length=2, 
                          nuc_cyto_ring=True, report=True)

In [ ]:
import napari
from skimage.color import gray2rgb

def well_search(base, row, column, marker):
    return base+'/'+[i for i in os.listdir(base) if (row + str(column).zfill(2) in i and marker in i and 'Track' in i)][0]

In [ ]:
res = pickle.load(open(well_search(base, 'C', 9, 'ERK'),'rb'))
well_search(base, 'C', 9, 'ERK')

In [ ]:
# trajectories
cells = [np.array(res.loc[res.particle==i, ['y', 'x']]) for i in np.random.choice(res.particle, 500)]
for i in range(500):
    plt.plot(cells[i][:,1], cells[i][:,0])
plt.show()

In [ ]:
cells = [np.array(res.loc[res.particle==i, '90_intensity']) for i in np.random.choice(res.particle, 500)]
cells = [i for i in cells if i.shape[0]==3]
plt.plot(np.mean(cells, 0), c='k')
plt.plot(np.transpose(cells[:50]), alpha=0.2)
plt.show()

In [ ]:
#doses = [100, 50, 10, 7.5, 5, 2.5, 1, 0.5, 0.1, 0.05]
wells = ['C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'C09']
well_intensities = {}
for j, well in enumerate(wells):
    print(well)
    res = pickle.load(open(well_search(base, 'C', j+2, 'ERK'), 'rb'))
    well_arr_cyto = md.stkread(Position=wells[j], Channel='Yellow')[:,:,:4]
    intensities = {}
    for p in np.unique(res.particle):
        if res.loc[(res.particle==p)&(res.frame.isin([0,1,2,3]))].shape[0]==4:
            intensities[p] = []
            for i in range(4):
                nuc_coords = np.array(res.loc[(res.particle==p)&(res.frame==i), 'nuc coords'])[0]
                cyto_coords = np.array(res.loc[(res.particle==p)&(res.frame==i), 'cyto coords'])[0]
                nuc_intensity = np.percentile(well_arr_cyto[nuc_coords[:,0], nuc_coords[:,1], i], 90)
                cyto_intensity = np.percentile(well_arr_cyto[cyto_coords[:,0], cyto_coords[:,1], i], 90)
                intensities[p].append(nuc_intensity/cyto_intensity)
    well_intensities[well] = intensities

In [ ]:
#well_arr_cyto = md.stkread(Position=conservative_wells[j], Channel='Yellow')
well_arr_cyto[:,:,[i for i in range(4,56)]].shape

In [ ]:
#doses = [100, 50, 10, 7.5, 5, 2.5, 1, 0.5, 0.1, 0.05]
conservative_wells = ['C02', 'C05', 'C06']
conservative_well_intensities = {}
for j, well in enumerate(conservative_wells):
    print(well)
    res = pickle.load(open(well_search(conservative_base, 'C', int(well[-1]), 'ERK'), 'rb'))
    well_arr_cyto = md.stkread(Position=conservative_wells[j], Channel='Yellow')[:,:,:4]
    intensities = {}
    for p in np.unique(res.particle):
        if res.loc[(res.particle==p)&(res.frame.isin([0,1,2,3]))].shape[0]==4:
            intensities[p] = []
            for i in range(4):
                nuc_coords = np.array(res.loc[(res.particle==p)&(res.frame==i), 'nuc coords'])[0]
                cyto_coords = np.array(res.loc[(res.particle==p)&(res.frame==i), 'cyto coords'])[0]
                nuc_intensity = np.percentile(well_arr_cyto[nuc_coords[:,0], nuc_coords[:,1], i], 90)
                cyto_intensity = np.percentile(well_arr_cyto[cyto_coords[:,0], cyto_coords[:,1], i], 90)
                intensities[p].append(nuc_intensity/cyto_intensity)
    conservative_well_intensities[well] = intensities

In [ ]:
cells = np.array(list(conservative_well_intensities['C02'].values()))
plt.plot(cells[np.random.choice(np.arange(cells.shape[0]), 50)].T)
plt.show()

In [ ]:
def cell_change(cell):
    return max(cell)/min(cell)

changes = {}
for j, well in enumerate(wells):
    cells = [cell_change(well_intensities[well][p]) for p in well_intensities[well].keys()]
    changes[str(well)] = cells
plt.rcParams['font.size'] = 14    
plt.figure(figsize=(8,6))
plt.title('Dose Response')
plt.boxplot(changes.values())
plt.ylabel('Max N/C Fold Change = Translocation')
plt.xlabel('Wells')
plt.xticks(np.arange(1,len(wells)+1), wells)
plt.ylim(0.9, 1.3)

plt.show()

In [ ]:
cons_well_changes = {}
for j, well in enumerate(conservative_wells):
    cells = [cell_change(conservative_well_intensities[well][p]) for p in conservative_well_intensities[well].keys()]
    cons_well_changes[str(well)] = cells
    
plt.title('Dose Response')
plt.boxplot([cons_well_changes['C02'], changes['C02'], cons_well_changes['C05'], changes['C05'], cons_well_changes['C06'], changes['C06']])
plt.ylabel('Max N/C Fold Change = Translocation')
plt.xlabel('Wells')
plt.xticks(np.arange(1,7), ['Cons. C02', 'C02', 'Cons. C05', 'C05', 'Cons. C06', 'C06'])
plt.ylim(0.9, 1.3)
plt.show()

In [ ]:
well_arr_cyto = md.stkread(Position='C02', Channel='Yellow')[:,:,:4]
well_arr_nuc = md.stkread(Position='C02', Channel='FarRed')[:,:,:4]
res = pickle.load(open(well_search(base, 'C', 2, 'ERK'), 'rb'))

In [ ]:
from segmentation import label_wvt, watershed_wvt
wvt_nuclear = label_wvt(well_arr_nuc)
watershed_wvt_nuclear = watershed_wvt(well_arr_nuc,wvt_nuclear)
res = pickle.load(open(well_search(base, 'C', 2, 'ERK'), 'rb'))


In [ ]:
cyto_coords = [np.concatenate(list(res.loc[res['frame']==i,'cyto coords'])) for i in range(4)]
nuc_cyto_movie = create_nuc_cyto_visualization(well_arr_cyto, watershed_wvt_nuclear, cyto_coords)

In [ ]:
adapt_nuc = [equalize_adapthist(well_arr_nuc[:,:,i]) for i in range(4)]

In [ ]:
nuc_movie = create_nuc_cyto_visualization(np.stack(adapt_nuc, -1), watershed_wvt_nuclear, cyto_coords)


In [ ]:
with napari.gui_qt():
    viewer = napari.view_image(nuc_movie, rgb='RGB')

In [ ]:
#res.loc[(res.particle==p)&(res.frame.isin([0,1,2]))].shape[0]==3
res.loc[res['frame']==0, 'nuc coords'][res.loc[res['frame']==0, 'nuc coords'].apply(lambda a: (594 in a) and (1620 in a))]

In [ ]:
# For Specific Cell Movies
cyto_coords = [np.concatenate(list(res.loc[(res['frame']==i)&(res['particle']==435),'cyto coords'])) for i in range(4)]
nuc_cyto_movie = create_nuc_cyto_visualization(well_arr_cyto, watershed_wvt_nuclear, cyto_coords)

In [ ]:
res.loc[(res['particle']==435), 'cyto coords'].values[0].shape


In [ ]:
cyto_435 = []
nuc_435 = []
cyto_672 = []
nuc_672 = []
for i in range(4):
    cyto_435.append(well_arr_cyto[res.loc[(res['particle']==435), 'cyto coords'].values[i][:,0], 
                                  res.loc[(res['particle']==435), 'cyto coords'].values[i][:,1], i])
    nuc_435.append(well_arr_cyto[res.loc[(res['particle']==435), 'nuc coords'].values[i][:,0], 
                                  res.loc[(res['particle']==435), 'nuc coords'].values[i][:,1], i])
    cyto_672.append(well_arr_cyto[res.loc[(res['particle']==672), 'cyto coords'].values[i][:,0], 
                                  res.loc[(res['particle']==672), 'cyto coords'].values[i][:,1], i])
    nuc_672.append(well_arr_cyto[res.loc[(res['particle']==672), 'nuc coords'].values[i][:,0], 
                                  res.loc[(res['particle']==672), 'nuc coords'].values[i][:,1], i])

In [ ]:
for i in range(4):
    plt.figure(figsize=(12,8))
    center = res.loc[(res['particle']==435), 'cyto coords'].values[i].mean(0).astype(np.int)
    plt.imshow(well_arr_cyto[center[0]-100:center[0]+100,center[1]-100:center[1]+100, i])
    ctemp = res.loc[(res['particle']==435), 'cyto coords'].values[i]
    plt.scatter(ctemp[:,1]-center[1]+100, ctemp[:,0]-center[0]+100, s=0.03, c='r')
    ntemp = res.loc[(res['particle']==435), 'nuc coords'].values[i]
    plt.scatter(ntemp[:,1]-center[1]+100, ntemp[:,0]-center[0]+100, s=0.03, c='r')
    plt.show()

In [ ]:
#plt.plot([np.percentile(cyto_435[i], 90) for i in range(4)])
#plt.plot([np.percentile(nuc_435[i], 90) for i in range(4)])
plt.plot([np.percentile(nuc_435[i], 90)/np.percentile(cyto_435[i], 90) for i in range(4)], label='90th%ile')
plt.plot([np.percentile(nuc_435[i], 50)/np.percentile(cyto_435[i], 50) for i in range(4)], label='50th%ile')
plt.plot([np.mean(nuc_435[i])/np.mean(cyto_435[i]) for i in range(4)], label='Mean')
plt.xlabel('Frames')
plt.ylabel('N/C Ratio')
plt.legend()
plt.show()


In [ ]:
for i in range(4):
    plt.figure(figsize=(12,8))
    center = res.loc[(res['particle']==672), 'cyto coords'].values[i].mean(0).astype(np.int)
    plt.imshow(well_arr_cyto[center[0]-100:center[0]+100,center[1]-100:center[1]+100, i])
    temp = res.loc[(res['particle']==672), 'cyto coords'].values[i]
    plt.scatter(temp[:,1]-center[1]+100, temp[:,0]-center[0]+100, s=0.05, c='r')
    plt.show()

In [ ]:
plt.plot([np.percentile(nuc_672[i], 90)/np.percentile(cyto_672[i], 90) for i in range(4)], label='90th%ile')
plt.plot([np.percentile(nuc_672[i], 50)/np.percentile(cyto_672[i], 50) for i in range(4)], label='50th%ile')
plt.plot([np.mean(nuc_672[i])/np.mean(cyto_672[i]) for i in range(4)], label='Mean')
plt.xlabel('Frames')
plt.ylabel('N/C Ratio')
plt.legend()
plt.show()

In [ ]:
plt.hist(res['cyto variability'], bins=1000, alpha=0.5)
plt.hist(res['nuc variability'], bins=1000, alpha=0.5)
#plt.ylim(0,100)
plt.show()


In [ ]:
j = 9
print('Dose:', doses[4])
res = pickle.load(open(well_search(base, 'C', j, 'ERK'), 'rb'))
print(well_search(base, 'C', j, 'ERK'))
options = [p for p in dose_intensities['C09'] if cell_change(dose_intensities['C09'][p])>1.14]
p = options[2]
print('Cell:', p, 'out of', len(options), 'options')
cell_check = []
for i in range(4):
    nuc_coords = np.array(res.loc[(res.particle==p)&(res.frame==i), 'nuc coords'])[0]
    cyto_coords = np.array(res.loc[(res.particle==p)&(res.frame==i), 'cyto coords'])[0]
    cell_check.append(create_nuc_cyto_visualization(well_arr_nuc, i, nuc_coords, cyto_coords))


In [ ]:
cell_frames = [equalize_adapthist(i) for i in cell_check]
with napari.gui_qt():
    viewer = napari.view_image(np.stack(cell_frames), rgb='RGB')

In [ ]:
#results
np.stack([np.array([well_arr_cyto[:,:,i]]) for i in range(4)], axis=-1).shape

In [ ]:
doses = [100, 50, 10, 7.5, 5, 2.5, 1, 0.5, 0.1, 0.05]
for i, dose in doses: 
    egf[dose] = 


In [ ]:
Row C: EGF
Row D: mTNFa
Row E: IL-3
Row F: IL-6
Concentrations are in descending order (C02 is 100 ng/uL):
egf = {100:, 50:, 10:, 7.5:, 5:, 2.5:, 1:, 0.5:, 0.1:, 0.05:}
mtfna = {100:, 50:, 10:, 7.5:, 5:, 2.5:, 1:, 0.5:, 0.1:, 0.05:}
il3 = {100:, 50:, 10:, 7.5:, 5:, 2.5:, 1:, 0.5:, 0.1:, 0.05:}
il6 = {100:, 50:, 10:, 7.5:, 5:, 2.5:, 1:, 0.5:, 0.1:, 0.05:}